In [1]:
import random
import numpy as np
import pandas as pd
from itertools import combinations

In [2]:
def get_int_value(card) -> int:
    """
    Returns the integer value of a card
    """
    if card == 'A':
        return 14
    elif card == 'K':
        return 13
    elif card == 'Q':
        return 12
    elif card == 'J':
        return 11
    elif card == 'T':
        return 10
    else:
        return int(card)

def get_str_value(card) -> str:
    """
    Returns the string value of a card
    """
    if card == 14:
        return 'A'
    elif card == 13:
        return 'K'
    elif card == 12:
        return 'Q'
    elif card == 11:
        return 'J'
    elif card == 10:
        return 'T'
    else:
        return str(card)

In [3]:
def check_hand(hand, board):
    """Returns a int with the value of hand, the value of the highest card in the hand and the value of the highest card in the board"""

    total = hand + board

    hand = pd.DataFrame([(x[0],x[1]) for x in hand], columns=['value', 'suit'])
    hand['int_value'] = hand.value.apply(get_int_value)
    
    total = pd.DataFrame([(x[0],x[1]) for x in total], columns=['value', 'suit'])
    total['int_value'] = total.value.apply(get_int_value)

    if is_straight_flush(total):
        return [8, total.int_value.max(), hand.int_value.max()]
    elif is_four_of_a_kind(total):
        return [7, total.int_value.value_counts().idxmax(), hand.int_value.max()]
    elif is_full_house(total):
        return [6, total.int_value.value_counts().idxmax(), hand.int_value.max()]
    elif is_flush(total):
        return [5, hand.int_value.max(), hand.int_value.max()]
    elif is_straight(total)[0]:
        return [4, total.int_value.max(), hand.int_value.max()]
    elif is_three_of_a_kind(total):
        return [3, total.int_value.value_counts().idxmax(), hand.int_value.max()]
    elif is_two_pair(total):
        return [2, total.int_value.value_counts()[total.int_value.value_counts() == 2].index.max(), hand.int_value.max()]
    elif is_one_pair(total):
        return [1, total.int_value.value_counts().idxmax(), hand.int_value.max()]
    else:
        return [0, total.int_value.max(), hand.int_value.max()]
    
# check if it is a straight flush
def is_straight_flush(hand):
    """Returns a boolean if it is a straight flush"""
    straight, hand = is_straight(hand)
    if straight and is_flush(hand):
        return True
    else:
        return False

# check if it is a four of a kind
def is_four_of_a_kind(hand) -> bool:
    """Returns a boolean if it is a four of a kind"""
    if hand.value.value_counts().max() == 4:
        return True
    else:
        return False
    
# check if it is a full house
def is_full_house(hand) -> bool:
    """Returns a boolean if it is a full house"""
    if 3 in hand.value.value_counts().values and 2 in hand.value.value_counts().values:
        return True
    else:
        return False

# check if it is a flush
def is_flush(hand) -> bool:
    """Returns a boolean if it is a flush"""
    if hand.suit.value_counts().values.max() >= 5:
        return True
    else:
        return False

# check if it is a straight
def is_straight(hand):
    """Returns a boolean if it is a straight"""
    # There are 7 cards in total to check
    # There can be a straight with 5 cards

    # Check if there is a straight with 5 cards
    perm = combinations(list(hand.value + hand.suit), 5)
    for i in list(perm):
        it_is, helper = is_straight_helper(i)
        if it_is:
            return True, helper
    return False, None

def is_straight_helper(hand):
    """ Returns a boolean if it is a straight with 5 cards"""
    hand = pd.DataFrame([(x[0],x[1]) for x in hand], columns=['value', 'suit'])
    hand['int_value'] = hand.value.apply(get_int_value)
    
    if hand.int_value.max() - hand.int_value.min() == 4 and len(set(hand.int_value)) == 5:
        return True, hand
    else:
        return False, None
    

# check if it is a three of a kind
def is_three_of_a_kind(hand) -> bool:
    """Returns a boolean if it is a three of a kind"""
    if 3 in hand.value.value_counts().values:
        return True
    else:
        return False

# check if it is a two pair
def is_two_pair(hand) -> bool:
    """Returns a boolean if it is a two pair"""
    if len(hand.value.value_counts()[hand.value.value_counts() == 2]) == 2 or len(hand.value.value_counts()[hand.value.value_counts() == 2]) == 3:
        return True
    else:
        return False

# check if it is a one pair
def is_one_pair(hand) -> bool:
    """Returns a boolean if it is a one pair"""
    if 2 in hand.value.value_counts().values:
        return True
    else:
        return False

# check if it is a high card
def is_high_card(hand) -> bool:
    """Returns a boolean if it is a high card"""
    if len(set(hand.value.value_counts().values)) == 7:
        return True
    else:
        return False

# check if it works
print(check_hand(['A♠', 'K♠'],['Q♠', 'J♠', 'T♠', 'J♦', '8♥'])) # straight flush
print(check_hand(['2♠', '2♦'],[ '2♥', '2♣', 'K♠', 'Q♠', 'J♠'])) # four of a kind
print(check_hand(['A♠', 'A♦'],[ 'A♥', 'K♠', 'K♦', '8♥', 'J♠'])) # full house
print(check_hand(['9♠', 'K♠'],[ 'Q♠', 'T♠', '2♠', 'J♦', '8♠'])) # flush
print(check_hand(['A♥', 'K♦'],[ 'Q♠', 'J♥', 'T♠', 'J♦', '8♥'])) # straight
print(check_hand(['A♠', 'A♦'],[ 'A♥', 'K♠', 'Q♠', 'J♦', '8♥'])) # three of a kind
print(check_hand(['A♠', 'A♦'],[ 'K♥', 'K♠', 'Q♠', 'J♦', '8♥'])) # two pair
print(check_hand(['A♠', 'A♦'],[ 'K♥', 'Q♠', 'J♦', '8♥', '7♠'])) # one pair
print(check_hand(['A♠', '2♦'],[ 'Q♥', 'J♠', 'T♦', '3♥', '7♠'])) # high card


[8, 14, 14]
[7, 2, 2]
[6, 14, 14]
[5, 13, 13]
[4, 14, 14]
[3, 14, 14]
[2, 14, 14]
[1, 14, 14]
[0, 14, 14]


In [4]:
def create_deck():
    """Create a deck of 52 cards"""
    return [x + y for x in ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2'] for y in ['♠', '♥', '♦', '♣']]

def shuffle_deck(cards):
    """Shuffle a deck of cards"""
    random.shuffle(cards)
    return cards

def deal_cards(cards, num_players):
    """Deal cards to all players"""
    hands = [[] for _ in range(num_players)]
    _ = [hands[j].append(cards.pop(0)) for i in range(2) for j in range(num_players)] 
    return hands, cards

def get_all_possibilities(cards, num_cards):
    """Get all possible combinations of cards"""
    if num_cards not in [1, 2, 3, 4, 5]:
        raise ValueError("Invalid number of cards")
    if num_cards == 1:
        return cards
    else:
        return list(combinations(cards, num_cards))       

def get_win_probabilities(hands, board):
    """Get the probability of winning for each player"""
    cards = create_deck()
    # Remove cards that are already dealt
    _ = [cards.remove(card) for hand in hands for card in hand]
    _ = [cards.remove(card) for card in board]
    
    # Simulate all possible outcomes
    total_length = len(hands[0]) + len(board)
    possibilities = get_all_possibilities(cards, 7 - total_length)

    # Get the probability of winning for each player
    results = []
    i = 0
    for possibility in possibilities:
        print(round(i / len(possibilities) * 100, 2), '%', end='\r')
        # Add the board cards to the possibility
        possibility = list(possibility) + board
        # Get the best hand for each player
        best_hands = [check_hand(list(possibility) + hand) for hand in hands]
        # Get the winner
        winner = best_hands.index(max(best_hands))
        results.append(winner)
        i += 1
    
    # Get the probability of winning for each player
    #results = pd.DataFrame(results, columns=['hand', 'high_value'])
    return results


In [11]:
hands

[['T♠', 'J♣'], ['3♠', '8♣']]

In [13]:
winner

1

In [9]:
total.index(11)

0

In [30]:
cards = create_deck()
cards = shuffle_deck(cards)
num_players = 2
hands, cards = deal_cards(cards, num_players)

boards = get_all_possibilities(cards, 5)
test = [[''.join(hands[0])+''.join(board), ''.join(hands[1])+''.join(board)] for board in boards]
df = pd.DataFrame(test, columns=['hand1', 'hand2'])
df['numbers1'] = df.hand1.apply(lambda x: x[::2])
df['numbers2'] = df.hand2.apply(lambda x: x[::2])
df['suits1'] = df.hand1.apply(lambda x: x[1::2])
df['suits2'] = df.hand2.apply(lambda x: x[1::2])

# get max card of each hand
max_hand = (max(get_int_value(hands[0][0][0]), get_int_value(hands[0][1][0])), max(get_int_value(hands[1][0][0]), get_int_value(hands[1][1][0])))
min_hand = (min(get_int_value(hands[0][0][0]), get_int_value(hands[0][1][0])), min(get_int_value(hands[1][0][0]), get_int_value(hands[1][1][0])))
if max_hand[0] == max_hand[1]:
    df['highest_hand'] = min_hand.index(max(min_hand)) + 1
else:
    df['highest_hand'] = max_hand.index(max(max_hand)) + 1
    

df['is_quads1'] = df.numbers1.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 4)
df['is_quads2'] = df.numbers2.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 4)
df['is_quads1'] = np.where(df.is_quads1, 7, 0)
df['is_quads2'] = np.where(df.is_quads2, 7, 0)

df['is_full1'] = df.numbers1.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 3 and min(x.count(x[0]), x.count(x[1])) == 2)
df['is_full2'] = df.numbers2.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 3 and min(x.count(x[0]), x.count(x[1])) == 2)
df['is_full1'] = np.where(df.is_full1, 6, 0)
df['is_full2'] = np.where(df.is_full2, 6, 0)

df['is_flush1'] = df.suits1.apply(lambda x: max(x.count(x[0]), x.count(x[1])) >= 5)
df['is_flush2'] = df.suits2.apply(lambda x: max(x.count(x[0]), x.count(x[1])) >= 5)
df['is_flush1'] = np.where(df.is_flush1, 5, 0)
df['is_flush2'] = np.where(df.is_flush2, 5, 0)

def is_straight(x):
    x = ''.join(sorted(x)).replace('T', 'R').replace('A','T').replace('R','A')
    for h in ['23456','34567','45678','56789','6789T','789TJ','9TJQK','TJQKA']:
        if h in x:
            return True
    return False
helper=['23456','34567','45678','56789','6789T','789TJ','9TJQK','TJQKA']
df['is_straight1'] = df.numbers1.apply(is_straight)
df['is_straight2'] = df.numbers2.apply(is_straight)
df['is_straight1'] = np.where(df.is_straight1, 4, 0)
df['is_straight2'] = np.where(df.is_straight2, 4, 0)

df['is_trips1'] = df.numbers1.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 3 and min(x.count(x[0]), x.count(x[1])) == 1)
df['is_trips2'] = df.numbers2.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 3 and min(x.count(x[0]), x.count(x[1])) == 1)
df['is_trips1'] = np.where(df.is_trips1, 3, 0)
df['is_trips2'] = np.where(df.is_trips2, 3, 0)

df['is_two_pair1'] = df.numbers1.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 2 and min(x.count(x[0]), x.count(x[1])) == 2)
df['is_two_pair2'] = df.numbers2.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 2 and min(x.count(x[0]), x.count(x[1])) == 2)
df['is_two_pair1'] = np.where(df.is_two_pair1, 2, 0)
df['is_two_pair2'] = np.where(df.is_two_pair2, 2, 0)

df['is_pair1'] = df.numbers1.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 2 and min(x.count(x[0]), x.count(x[1])) == 1)
df['is_pair2'] = df.numbers2.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 2 and min(x.count(x[0]), x.count(x[1])) == 1)
df['is_pair1'] = np.where(df.is_pair1, 1, 0)
df['is_pair2'] = np.where(df.is_pair2, 1, 0)

df['is_high1'] = df.numbers1.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 1)
df['is_high2'] = df.numbers2.apply(lambda x: max(x.count(x[0]), x.count(x[1])) == 1)
df['is_high1'] = np.where(df.is_high1, 0, 0)
df['is_high2'] = np.where(df.is_high2, 0, 0)

df['max_value1'] = df[[x for x in df.columns if 'is_' in x and '1' in x]].max(axis=1)
df['max_value2'] = df[[x for x in df.columns if 'is_' in x and '2' in x]].max(axis=1)

df['winner'] = df['highest_hand']
df['winner'] = np.where(df['max_value1'] > df['max_value2'], 1, df['winner'])
df['winner'] = np.where(df['max_value1'] < df['max_value2'], 2, df['winner'])


results = df['winner'].value_counts().sort_index() / df.shape[0] * 100

print('Hand 1: ', ''.join(hands[0]), 'Win probabilities: ', round(results.loc[1], 2))
print('Hand 2: ', ''.join(hands[1]), 'Win probabilities: ', round(results.loc[2], 2))

Hand 1:  T♣7♠ Win probabilities:  30.7
Hand 2:  K♣8♣ Win probabilities:  69.3
